In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [6]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [7]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [8]:
sample_query_cost = 1193.88
sample_query_timing = 185.896

In [9]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [10]:
%sql \d halloffame

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
9 rows affected.


Column,Type,Modifiers
playerid,character varying(10),not null
yearid,integer,not null
votedby,character varying(64),not null default ''::character varying
ballots,integer,
needed,integer,
votes,integer,
inducted,character varying(1),default NULL::character varying
category,character varying(20),default NULL::character varying
needed_note,character varying(20),default NULL::character varying


In [11]:
%%sql
SELECT * FROM People LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,deathcountry,deathstate,deathcity,namefirst,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
aaronha01,1934,2,5,USA,AL,Mobile,None,None,None,None,None,None,Hank,Aaron,Henry Louis,180,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
aasedo01,1954,9,8,USA,CA,Orange,None,None,None,None,None,None,Don,Aase,Donald William,190,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
abadan01,1972,8,25,USA,FL,Palm Beach,None,None,None,None,None,None,Andy,Abad,Fausto Andres,184,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [12]:
%%sql result_1a <<
SELECT namefirst, namelast, p.playerid, yearid
FROM halloffame as h, People as p
WHERE h.playerid = p.playerid AND h.inducted = 'Y';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [13]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [14]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [15]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [167]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=10.516..11.297 rows=323 loops=1)
   Hash Cond: ((halloffame.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.022..0.651 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=10.280..10.282 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.019..5.262 rows=19370 loops=1)
 Planning Time: 1.109 ms
 Execution Time: 11.433 ms
(10 rows)



In [168]:
inducted_hof_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM halloffame as h, People as p WHERE h.playerid = p.playerid AND h.inducted = 'Y';"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=11.264..11.999 rows=323 loops=1)
   Hash Cond: ((h.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=323 width=13) (actual time=0.025..0.610 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=10.966..10.967 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.024..5.434 rows=19370 loops=1)
 Planning Time: 1.152 ms
 Execution Time: 12.153 ms
(10 rows)



In [169]:
provided_query_cost = 959.06
provided_query_timing = 11.474
your_query_cost = 959.06
your_query_timing = 11.495

In [170]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [20]:
q1b_part2 = ['A', 'F']

In [21]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [22]:
%sql \d collegeplaying

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
3 rows affected.


Column,Type,Modifiers
playerid,character varying(10),not null
schoolid,character varying(15),not null
yearid,integer,not null


In [23]:
%%sql
SELECT * FROM collegeplaying LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,schoolid,yearid
aardsda01,pennst,2001
aardsda01,rice,2002
aardsda01,rice,2003
abadan01,gamiddl,1992
abadan01,gamiddl,1993


In [24]:
%%sql
SELECT * FROM schools LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


schoolid,schoolname,schoolcity,schoolstate,schoolnick
abilchrist,Abilene Christian University,Abilene,TX,USA
adelphi,Adelphi University,Garden City,NY,USA
adrianmi,Adrian College,Adrian,MI,USA
akron,University of Akron,Akron,OH,USA
alabama,University of Alabama,Tuscaloosa,AL,USA


In [25]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
SELECT namefirst, namelast, p.playerid, s.schoolid, h.yearid
FROM collegeplaying as c, halloffame as h, People as p, schools as s
WHERE h.playerid = p.playerid AND h.playerid = c.playerid AND c.schoolid = s.schoolid AND h.inducted = 'Y' AND s.schoolstate = 'CA'
ORDER BY h.yearid;
SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [26]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [27]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [28]:
%%sql with_view <<
SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [29]:
%%sql without_view <<
SELECT s.schoolid, h.yearid, COUNT(*)
FROM collegeplaying as c, halloffame as h, People as p, schools as s
WHERE h.playerid = p.playerid AND h.playerid = c.playerid AND c.schoolid = s.schoolid AND h.inducted = 'Y' AND s.schoolstate = 'CA'
GROUP BY s.schoolid, h.yearid
ORDER BY h.yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [30]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [31]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [32]:
with_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca GROUP BY schoolid, yearid ORDER BY yearid"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=525.47..527.39 rows=96 width=20) (actual time=6.237..6.250 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Sort  (cost=525.47..525.71 rows=96 width=12) (actual time=6.228..6.233 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=521.11..522.31 rows=96 width=12) (actual time=6.179..6.188 rows=23 loops=1)
               ->  Sort  (cost=521.11..521.35 rows=96 width=286) (actual time=6.177..6.183 rows=23 loops=1)
                     Sort Key: h.yearid
                    

In [33]:
without_view_query_str = "SELECT s.schoolid, h.yearid, COUNT(*) FROM collegeplaying as c, halloffame as h, People as p, schools as s WHERE h.playerid = p.playerid AND h.playerid = c.playerid AND c.schoolid = s.schoolid AND h.inducted = 'Y' AND s.schoolstate = 'CA' \
GROUP BY s.schoolid, h.yearid ORDER BY h.yearid;"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                       QUERY PLAN                                                                        
---------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=521.11..523.03 rows=96 width=20) (actual time=5.714..5.726 rows=13 loops=1)
   Group Key: h.yearid, s.schoolid
   ->  Sort  (cost=521.11..521.35 rows=96 width=12) (actual time=5.707..5.711 rows=23 loops=1)
         Sort Key: h.yearid, s.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=386.71..517.95 rows=96 width=12) (actual time=5.198..5.647 rows=23 loops=1)
               Join Filter: ((h.playerid)::text = (p.playerid)::text)
               ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=5.165..5.519 rows=23 loops=1)
                     Hash Cond: ((h.playerid)::text = (c.playerid)::text)
             

In [34]:
with_view_cost = 527.39
with_view_timing = 5.999
without_view_cost = 523.03
without_view_timing = 5.906

In [35]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [36]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
SELECT namefirst, namelast, p.playerid, s.schoolid, h.yearid
FROM collegeplaying as c, halloffame as h, People as p, schools as s
WHERE h.playerid = p.playerid AND h.playerid = c.playerid AND c.schoolid = s.schoolid AND h.inducted = 'Y' AND s.schoolstate = 'CA'
ORDER BY h.yearid;
SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [37]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [38]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [39]:
%%sql with_materialized_view <<
SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca_mat
GROUP BY schoolid, yearid
ORDER BY yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [40]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [41]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [42]:
with_materialized_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca_mat \
GROUP BY schoolid, yearid \
ORDER BY yearid"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.068..0.070 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.024..0.028 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=52) (actual time=0.005..0.007 rows=23 loops=1)
 Planning Time: 0.398 ms
 Execution Time: 0.191 ms
(9 rows)



In [43]:
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.197

In [44]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [45]:
q1e_part4 = ['D', 'J']

In [46]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

The MV is faster than View because MV is stored on the disk. So, Choose D and J that MV has a lower cost and execution time, but more time to create (MV should be cached when created but Views are computed on demand). And therefore, their query plan is different that Views have to merge and calculate a bunch of stuff but MV just executes order by and group by, so not choose G.

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [47]:
query_view_str = "SELECT * FROM inducted_hof_ca"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                    QUERY PLAN                                                                     
---------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=530.54..530.78 rows=96 width=33) (actual time=6.139..6.145 rows=23 loops=1)
   Sort Key: h.yearid
   Sort Method: quicksort  Memory: 26kB
   ->  Nested Loop  (cost=386.71..527.38 rows=96 width=33) (actual time=5.519..6.101 rows=23 loops=1)
         Join Filter: ((h.playerid)::text = (p.playerid)::text)
         ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=5.457..5.824 rows=23 loops=1)
               Hash Cond: ((h.playerid)::text = (c.playerid)::text)
               ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=323 width=13) (actual time=0.029..0.615 rows=323 loops=1)
                     Filter: ((inducted)::text = 'Y'::text)
                     Rows R

In [48]:
query_view_cost = 530.78
query_view_timing = 7.133

In [49]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [50]:
%%sql result_2b <<
SELECT * FROM inducted_hof_ca
WHERE yearid > 2010

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [51]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca WHERE yearid > 2010"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=209.92..209.93 rows=6 width=33) (actual time=0.982..0.984 rows=8 loops=1)
   Sort Key: h.yearid
   Sort Method: quicksort  Memory: 25kB
   ->  Nested Loop  (cost=0.85..209.84 rows=6 width=33) (actual time=0.582..0.950 rows=8 loops=1)
         Join Filter: ((h.playerid)::text = (p.playerid)::text)
         ->  Nested Loop  (cost=0.56..207.24 rows=6 width=30) (actual time=0.544..0.858 rows=8 loops=1)
               ->  Nested Loop  (cost=0.29..190.63 rows=55 width=30) (actual time=0.528..0.746 rows=27 loops=1)
                     ->  Seq Scan on halloffame h  (cost=0.00..106.86 rows=21 width=13) (actual time=0.491..0.536 rows=31 loops=1)
          

In [52]:
query_view_with_filter_cost = 209.93
query_view_with_filter_timing = 1.051

In [53]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [54]:
q2c = ['A', 'E']

In [55]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

Adding a filter reduce the pages that need to be read by the system, so it lowers the cost and excutive time.

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [56]:
%%sql q3a_join <<
SELECT * FROM people p, collegeplaying c
WHERE p.playerid = c.playerid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [57]:
q3a_query_str = 'SELECT * FROM people p, collegeplaying c WHERE p.playerid = c.playerid'
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..1193.88 rows=17350 width=167) (actual time=9.524..19.935 rows=17350 loops=1)
   Hash Cond: ((c.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying c  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.025..1.469 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=9.257..9.260 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.026..2.557 rows=19370 loops=1)
 Planning Time: 1.293 ms
 Execution Time: 20.739 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [58]:
q3a = ['C']

In [59]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [60]:
%%sql q3b_join <<
SELECT * FROM people p, collegeplaying c
WHERE p.playerid = c.playerid
ORDER BY p.playerid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [61]:
q3b_query_str = "SELECT * FROM people p, collegeplaying c WHERE p.playerid = c.playerid ORDER BY p.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                         QUERY PLAN                                                                         
------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..1910.20 rows=17350 width=167) (actual time=0.053..18.259 rows=17350 loops=1)
   Merge Cond: ((p.playerid)::text = (c.playerid)::text)
   ->  Index Scan using master_pkey on people p  (cost=0.29..1024.36 rows=19370 width=146) (actual time=0.024..4.267 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying c  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.023..2.323 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 1.438 ms
 Execution Time: 18.978 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [62]:
q3b = ['B']

In [63]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [64]:
%%sql q3c_join <<
SELECT * FROM people p1, people p2
WHERE p1.playerid != p2.playerid
LIMIT 1000

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


Inspect the query plan for the query you wrote above.

In [65]:
q3c_query_str = "SELECT * FROM people p1, people p2 WHERE p1.playerid != p2.playerid LIMIT 1000"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..15.00 rows=1000 width=292) (actual time=0.052..1.039 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..5629241.33 rows=375177530 width=292) (actual time=0.051..0.969 rows=1000 loops=1)
         Join Filter: ((p1.playerid)::text <> (p2.playerid)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people p1  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.024..0.024 rows=1 loops=1)
         ->  Materialize  (cost=0.00..716.55 rows=19370 width=146) (actual time=0.019..0.526 rows=1001 loops=1)
               ->  Seq Scan on people p2  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.001..0.112 rows=1001 loops=1)
 Planning Time: 0.813 ms
 Execution Time: 1.186 ms
(9 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [66]:
q3c = ['A']

In [67]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

When the merge has equi join constrain and data is sorted, the query will conduct with merge join which is fastest; 

When the merge has equi join constrain but data is not sorted, then with hash join, becasue hash join is the second fastest;

If it is a non-equijoin constrain, merge and hash join is not applicable, so the query choose nested-loops.

One benefit of merge join is that the output is also sorted.

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [68]:
%%sql result_4a <<
SELECT a.playerid, AVG(salary) salary
FROM appearances a NATURAL JOIN salaries s
WHERE a.g_batting = 10
GROUP BY a.playerid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [69]:
%sql SELECT * FROM appearances LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


yearid,teamid,lgid,playerid,g_all,gs,g_batting,g_defense,g_p,g_c,g_1b,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
1871,TRO,NA,abercda01,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1871,RC1,NA,addybo01,25,25,25,25,0,0,0,22,0,3,0,0,0,0,0,0,0
1871,CL1,NA,allisar01,29,29,29,29,0,0,0,2,0,0,0,29,0,29,0,0,0
1871,WS3,NA,allisdo01,27,27,27,27,0,27,0,0,0,0,0,0,0,0,0,0,0
1871,RC1,NA,ansonca01,25,25,25,25,0,5,1,2,20,0,1,0,0,1,0,0,0


In [70]:
%sql SELECT * FROM salaries LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


yearid,teamid,lgid,playerid,salary
1985,ATL,NL,barkele01,870000.0
1985,ATL,NL,bedrost01,550000.0
1985,ATL,NL,benedbr01,545000.0
1985,ATL,NL,campri01,633333.0
1985,ATL,NL,ceronri01,625000.0


In [71]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [72]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [73]:
result_4a_query_str = "SELECT a.playerid, AVG(salary) salary \
FROM appearances a NATURAL JOIN salaries s \
WHERE a.g_batting = 10 \
GROUP BY a.playerid"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3634.86..3634.89 rows=1 width=17) (actual time=23.341..23.413 rows=134 loops=1)
   Group Key: a.playerid
   ->  Sort  (cost=3634.86..3634.87 rows=1 width=17) (actual time=23.327..23.341 rows=138 loops=1)
         Sort Key: a.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2898.08..3634.85 rows=1 width=17) (actual time=15.101..23.143 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..45

In [74]:
result_4a_cost = 3634.89
result_4a_timing = 23.570

In [75]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [76]:
%%sql result_4b <<
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [77]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2346.09..2346.11 rows=1 width=17) (actual time=11.687..11.769 rows=134 loops=1)
   Group Key: a.playerid
   ->  Sort  (cost=2346.09..2346.09 rows=1 width=17) (actual time=11.674..11.685 rows=138 loops=1)
         Sort Key: a.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1609.30..2346.08 rows=1 width=17) (actual time=3.479..11.462 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..459

In [78]:
result_4b_cost = 2346.11
result_4b_timing = 11.917

In [79]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [80]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [81]:
%%sql result_4c <<
CREATE INDEX salary_idx
ON salaries (salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [82]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3634.86..3634.89 rows=1 width=17) (actual time=23.431..23.500 rows=134 loops=1)
   Group Key: a.playerid
   ->  Sort  (cost=3634.86..3634.87 rows=1 width=17) (actual time=23.418..23.428 rows=138 loops=1)
         Sort Key: a.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2898.08..3634.85 rows=1 width=17) (actual time=15.163..23.230 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..45

In [83]:
result_4c_cost = 3634.89
result_4c_timing = 23.617

In [84]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [85]:
q4d_part1 = ['D', 'E']

In [86]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

The g_batting is in the WHERE clause, and index it can save a lot of time and cost because we only need to look over tuples that satisfy g_batting = 10. However, index salary does not help because we do not need to lookup for salary.

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [87]:
%%sql 
DROP INDEX salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [88]:
%%sql result_4ei <<
SELECT a.playerid, a.yearid, s.salary
FROM appearances a NATURAL JOIN salaries s
WHERE a.g_batting = 10 and a.g_all = 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [89]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [90]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [91]:
result_4ei_query_str = "SELECT a.playerid, a.yearid, s.salary \
FROM appearances a NATURAL JOIN salaries s \
WHERE a.g_batting = 10 and a.g_all = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3279.47 rows=1 width=21) (actual time=9.802..17.372 rows=120 loops=1)
   ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=18 width=20) (actual time=0.017..14.024 rows=1289 loops=1)
         Filter: ((g_batting = 10) AND (g_all = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries s  (cost=0.29..8.09 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
         Index Cond: ((yearid = a.yearid) AND ((teamid)::text = (a.teamid)::text) AND ((lgid)::text = (a.lgid)::text) AND ((playerid)::text = (a.playerid)::text))
 Planning Time: 1.744 ms
 Execution Time: 17.443 ms
(8 rows)



In [92]:
result_4ei_cost = 3279.47
result_4ei_timing = 17.443

In [93]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [94]:
%%sql result_4eii << 
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [95]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=17.91..1732.85 rows=1 width=21) (actual time=3.101..6.161 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances a  (cost=17.63..1587.23 rows=18 width=20) (actual time=0.340..2.851 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on g_batting_idx  (cost=0.00..17.62 rows=1244 width=0) (actual time=0.212..0.213 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salaries_pkey on salaries s  (cost=0.29..8.09 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
     

In [96]:
result_4eii_with_index_cost = 1732.85
result_4eii_with_index_timing = 6.260

In [97]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [98]:
%%sql 
DROP INDEX g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [99]:
%%sql result_4eiii <<
SELECT a.playerid, a.yearid, s.salary
FROM appearances a NATURAL JOIN salaries s
WHERE a.g_batting = 10 or a.g_all = 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [100]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [101]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [102]:
result_4eiii_query_str = "SELECT a.playerid, a.yearid, s.salary \
FROM appearances a NATURAL JOIN salaries s \
WHERE a.g_batting = 10 or a.g_all = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3187.98..3924.75 rows=1 width=21) (actual time=16.429..24.503 rows=216 loops=1)
   Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.019..2.173 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2707 width=20) (actual time=16.193..16.195 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 121kB
         ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=2707 width=20) (actual time=0.016..15.423 rows=1655 loops=1)
      

In [103]:
result_4eiii_cost = 3924.75
result_4eiii_timing = 24.591

In [104]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [105]:
%%sql result_4eiv << 
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [106]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3187.98..3924.75 rows=1 width=21) (actual time=16.030..24.210 rows=216 loops=1)
   Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.018..2.230 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2707 width=20) (actual time=15.762..15.766 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 121kB
         ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=2707 width=20) (actual time=0.015..15.050 rows=1655 loops=1)
      

In [107]:
result_4eiv_with_index_cost = 3924.75
result_4eiv_with_index_timing = 24.304

In [108]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [109]:
%%sql 
DROP INDEX g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [110]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx
ON appearances (g_batting, g_all);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [111]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2892.77..3629.54 rows=1 width=21) (actual time=4.777..12.621 rows=216 loops=1)
   Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.015..2.087 rows=26428 loops=1)
   ->  Hash  (cost=2838.63..2838.63 rows=2707 width=20) (actual time=4.573..4.578 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 121kB
         ->  Bitmap Heap Scan on appearances a  (cost=1181.19..2838.63 rows=2707 width=20) (actual time=1.340..4.058 rows=1655 loops=1)

In [112]:
result_4ev_multiple_col_index_cost = 3629.54
result_4ev_multiple_col_index_timing = 12.569

In [113]:
grader.check("q4ev")

q4ev results: All test cases passed!

In [114]:
%%sql
DROP INDEX g_batting_g_all_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [115]:
q4e_part6 = ['C', 'I']

In [116]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

With index on a column used in an AND predicate, the query will only look over tuples that satisfy constraints instead of scanning all tuples, so choose C that reduce time and cost.

With index on a column used in an OR predicate, the query will, like without any index, scan all tuples, so the cost and time basically is the same as those with no index, and therefore D E F is incorrect.

With multicolumn index on a column used in an OR predicate, the query will scan tuples based on the two constraints together, and index is being used in a hash merge, so it helps reduce pages that need examination and reduce cost and time. Choose I.



<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [117]:
%%sql result_5a_min << 
SELECT min(salary)
FROM salaries

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [118]:
result_5a_min_query_str = "SELECT min(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.771..4.772 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.015..2.205 rows=26428 loops=1)
 Planning Time: 0.572 ms
 Execution Time: 4.852 ms
(4 rows)



In [119]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 4.852

In [120]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [121]:
%%sql result_5a_avg <<
SELECT avg(salary)
FROM salaries

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [122]:
result_5a_avg_query_str = "SELECT avg(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.087..5.089 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.013..2.092 rows=26428 loops=1)
 Planning Time: 0.410 ms
 Execution Time: 5.162 ms
(4 rows)



In [123]:
result_5a_avg_query_cost = 525.36
result_5a_avg_query_timing = 5.162

In [124]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [125]:
%%sql 
CREATE INDEX salary_idx
ON salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [126]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                  QUERY PLAN                                                                  
----------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.062..0.063 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.060..0.060 rows=1 loops=1)
           ->  Index Only Scan using salary_idx on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.058..0.059 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.672 ms
 Execution Time: 0.128 ms
(8 rows)



In [127]:
result_5b_min_query_cost = 0.33
result_5b_min_query_timing = 0.116

In [128]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [129]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.129..5.130 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.015..2.144 rows=26428 loops=1)
 Planning Time: 0.506 ms
 Execution Time: 5.230 ms
(4 rows)



In [130]:
result_5b_avg_query_cost = 525.36
result_5b_avg_query_timing = 5.230

In [131]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [132]:
%%sql 
DROP INDEX salary_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [133]:
q5c = ['D']

In [134]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

For min and max, index is good at searching for the smallest or largest value in a B+ tree, but AVG and Count have to scan all tuples so they will not benefit from index.

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [135]:
%%sql result_6a <<
SELECT a.playerid, a.yearid, a.teamid, ab
FROM appearances a NATURAL JOIN batting
WHERE ab > 500;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [136]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [137]:
pd.read_csv("results/result_6a.csv")

,playerid,yearid,teamid,ab
0,aaronha01,1955,ML1,602
1,aaronha01,1956,ML1,609
2,aaronha01,1957,ML1,615
3,aaronha01,1958,ML1,601
4,aaronha01,1959,ML1,629
...,...,...,...,...
995,brownji03,1942,SLN,606
996,brownol02,1969,SDN,568
997,brownol02,1970,SDN,534
998,brownpe01,1887,LS2,547


In [138]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [139]:
result_6a_query_str = "SELECT a.playerid, a.yearid, a.teamid, ab \
FROM appearances a NATURAL JOIN batting \
WHERE ab > 500;"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                                                            QUERY PLAN                                                                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Gather  (cost=4409.81..6812.50 rows=7 width=21) (actual time=35.341..53.566 rows=8839 loops=1)
   Workers Planned: 1
   Workers Launched: 1
   ->  Parallel Hash Join  (cost=3409.81..5811.80 rows=4 width=21) (actual time=32.399..44.606 rows=4420 loops=2)
         Hash Cond: ((batting.yearid = a.yearid) AND ((batting.teamid)::text = (a.teamid)::text) AND ((batting.lgid)::text = (a.lgid)::text) AND ((batting.playerid)::text = (a.playerid)::text))
         ->  Parallel Seq Scan on batting  (cost=0.00..2347.09 rows=5229 width=24) (actual time=0.187..8.668 rows=4420 loops=2)
               Filter: (ab > 500)

In [140]:
result_6a_cost = 6812.50
result_6a_timing = 54.007

In [141]:
grader.check("q6aii")

q6aii results: All test cases passed!

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [142]:
%%sql 
CLUSTER batting USING batting_pkey

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [143]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                                                            QUERY PLAN                                                                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Gather  (cost=4409.81..6806.50 rows=7 width=21) (actual time=35.954..54.499 rows=8839 loops=1)
   Workers Planned: 1
   Workers Launched: 1
   ->  Parallel Hash Join  (cost=3409.81..5805.80 rows=4 width=21) (actual time=33.097..45.697 rows=4420 loops=2)
         Hash Cond: ((batting.yearid = a.yearid) AND ((batting.teamid)::text = (a.teamid)::text) AND ((batting.lgid)::text = (a.lgid)::text) AND ((batting.playerid)::text = (a.playerid)::text))
         ->  Parallel Seq Scan on batting  (cost=0.00..2341.09 rows=5229 width=24) (actual time=0.020..8.923 rows=4420 loops=2)
               Filter: (ab > 500)

In [144]:
result_6b_cost = 6806.50
result_6b_timing = 54.944

In [145]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [146]:
%%sql 
CREATE INDEX ab_idx ON batting(ab);
CLUSTER batting USING ab_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [147]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                                                            QUERY PLAN                                                                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Gather  (cost=4511.00..6206.97 rows=7 width=21) (actual time=34.352..44.242 rows=8839 loops=1)
   Workers Planned: 1
   Workers Launched: 1
   ->  Parallel Hash Join  (cost=3511.00..5206.27 rows=4 width=21) (actual time=31.530..36.091 rows=4420 loops=2)
         Hash Cond: ((batting.yearid = a.yearid) AND ((batting.teamid)::text = (a.teamid)::text) AND ((batting.lgid)::text = (a.lgid)::text) AND ((batting.playerid)::text = (a.playerid)::text))
         ->  Parallel Bitmap Heap Scan on batting  (cost=101.19..1741.56 rows=5229 width=24) (actual time=0.398..1.127 rows=4420 loops=2)
               Recheck 

In [148]:
result_6c_cost = 6206.97
result_6c_timing = 44.695

In [149]:
grader.check("q6c")

q6c results: All test cases passed!

In [150]:
%sql DROP INDEX ab_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [151]:
q6d = ['A','D','E','G']

In [152]:
grader.check("q6d")

q6d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

The query filters on ab so ab_idx rather than batting_pkey so ab_idx works to a lower cost and time. Plus, batting_pkey does decrease the cost because scan a clustered index structue is a bit lighter than table scan, but it can also lead to a higher execution time due to the penalty of uncessary index.

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [153]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [154]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.6 ms, sys: 2.34 ms, total: 5.94 ms
Wall time: 3.42 s


[]

In [155]:
result_7a_timing = 3.42

In [156]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [157]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [159]:
%%sql 
CREATE INDEX salary_idx
ON salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [160]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 5.75 ms, sys: 820 µs, total: 6.57 ms
Wall time: 6.08 s


[]

In [161]:
result_7b_timing = 6.08

In [162]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

When I add an index into the salaries table the wall time increased. I suppose that is because new tuples are not reflected in index and the incomplete index runs relatively slow.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

Join: Basically there are 3 types of join, merge join is the fastest, then hash join, and the slowest is nested loop join. When the merge has equi join constrain and data is sorted, like after ORDER BY, the query optimizer will choose merge join. When the merge has equi join constrain but data is not sorted, then with hash join, becasue hash join is the second fastest; If it is a non-equijoin constrain, like "!=", merge and hash joins are not applicable, so the optimizer has to choose nested-loops.

Index: Index helps in speeding up the queries by providing swift access to the requested data instead of scanning the whole table to retrieve a few tuples, and B+ trees is a popular algorithm to find those tuples. Whereas, index also get some constrains, like if the feature has too many duplictaes index will not help much, and also index is difficilt for long-term maintenance due to low performance od the incomplete index.

Clustered Index: A clustered index is useful for improving query performance because the data is sorted on disk, which indicates they have high efficiency when we search for a value or a subset. We usually use key attributes or attributes often used in WHERE clause to cluster. However, if the cluster index does not point to the attributes that query plan to sort or filter, then the index will have little benefit.


<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [172]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

updating: results/ (stored 0%)
updating: results/result_1a.csv (deflated 53%)
updating: results/result_1c.csv (deflated 64%)
updating: results/result_1di_view.csv (deflated 37%)
updating: results/result_1di_no_view.csv (deflated 37%)
updating: results/result_1ei.csv (deflated 64%)
updating: results/result_1eii.csv (deflated 37%)
updating: results/result_4a.csv (deflated 59%)
updating: results/result_4ei.csv (deflated 62%)
updating: results/result_4eiii.csv (deflated 64%)
updating: results/result_6a.csv (deflated 72%)


/home/jovyan/fa22/proj/proj2/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()